In [1]:
import pandas as pd
from pyspark.sql.functions import col,split,concat,concat_ws,size
import re
import pyspark
import geopandas
import matplotlib.pyplot as plt

In [2]:
ds = sc.textFile("s3://mukuawsproject/sample_geo.txt")
rdd=ds.map(lambda x: (re.split('[\t]',x))).cache()
rdd=rdd.filter(lambda x:len(x)!=1)
rdd=rdd.filter(lambda x:x[0]!='Latitude')
rdd=rdd.map(lambda x:(float(x[0]),float(x[1]),float(x[2])))


In [3]:
dfx = sqlContext.createDataFrame(data=rdd)


In [ ]:
dfx.show(10)

In [ ]:
dfx=dfx.select(concat('_1').alias("latitude"),concat('_2').alias("longitude"),concat('_3').alias("locationID"))

In [ ]:
dfx.show(10)

In [ ]:
dfx=dfx.filter(dfx.latitude!=0)
dfx=dfx.filter(dfx.longitude!=0)

In [ ]:
dfx.show(10)

In [ ]:
dfx=dfx.toPandas();
dfx.to_csv('s3://mukuawsproject/sample_geo.csv',header=None)

In [ ]:
gdf = geopandas.GeoDataFrame(
    dfx, geometry=geopandas.points_from_xy(dfx.longitude, dfx.latitude))


world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# We restrict to South America.
ax = world.plot(
    edgecolor='black',figsize=(16,16))

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax,color='red')

plt.show()